# 数据可视化


In [2]:
import os
import pandas as pd
import warnings
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import plotly_express as px
import plotly as py
import plotly.io as pio
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

py.offline.init_notebook_mode(connected=True)
tqdm.pandas()


## 读取数据

### 读取业务指标

#### 正常数据

In [25]:
service_metric_data1 = pd.read_csv(
    '../data/training_data_normal/cloudbed-1/metric/service/metric_service.csv')
# service_metric_data2 = pd.read_csv(
#     '../data/training_data_normal/cloudbed-2/metric/service/metric_service.csv')
# service_metric_data3 = pd.read_csv(
#     '../data/training_data_normal/cloudbed-3/metric/service/metric_service.csv')


#### 异常数据

In [28]:
service_metric_data1 = pd.read_csv(
    '../data/training_data_with_faults/tar/2022-03-20-cloudbed1/metric/service/metric_service.csv')
service_metric_data2 = pd.read_csv(
    '../data/training_data_with_faults/tar/2022-03-21-cloudbed1/metric/service/metric_service.csv')
# service_metric_data3 = pd.read_csv(
#     '../data/training_data_with_faults/tar/2022-03-20-cloudbed2/metric/service/metric_service.csv')
# service_metric_data4 = pd.read_csv(
#     '../data/training_data_with_faults/tar/2022-03-21-cloudbed2/metric/service/metric_service.csv')
# service_metric_data5 = pd.read_csv(
#     '../data/training_data_with_faults/tar/2022-03-20-cloudbed3/metric/service/metric_service.csv')
# service_metric_data6 = pd.read_csv(
#     '../data/training_data_with_faults/tar/2022-03-21-cloudbed3/metric/service/metric_service.csv')
# service_metric_data7 = pd.read_csv(
#     '../data/training_data_with_faults/tar/2022-03-24-cloudbed3/metric/service/metric_service.csv')


In [29]:
# service_metric_data=pd.concat([service_metric_data1,service_metric_data2,service_metric_data3])
# service_metric_data = pd.concat(
#     [service_metric_data1, service_metric_data2, service_metric_data3, service_metric_data4, service_metric_data5,service_metric_data6,service_metric_data7])
service_metric_data = pd.concat(
    [service_metric_data1, service_metric_data2])
service_metric_data


,service,timestamp,rr,sr,mrt,count
0,adservice-grpc,1647716400,100.0,100.0,2.429508,61
1,adservice-grpc,1647716460,100.0,100.0,2.429508,61
2,adservice-grpc,1647716520,100.0,100.0,2.332967,91
3,adservice-grpc,1647716580,100.0,100.0,2.647015,67
4,adservice-grpc,1647716640,100.0,100.0,2.510000,85
...,...,...,...,...,...,...
15835,frontend-http,1647820500,100.0,100.0,53.037687,134
15836,frontend-http,1647820560,100.0,100.0,247.060484,124
15837,frontend-http,1647820620,100.0,100.0,549.454167,120
15838,frontend-http,1647820680,100.0,100.0,883.313559,118


In [27]:
processed_service_metric_data = list(service_metric_data.groupby('service'))
for (service,service_data) in tqdm(processed_service_metric_data):
    service_data.sort_values(by='timestamp', inplace=True)
    service_data.reset_index(drop=True, inplace=True)
    service_data['datetime'] = pd.to_datetime(
        service_data['timestamp'], unit='s')
    
    # processed_data_path = f'../data/training_data_normal/processed_service_metric_data/'
    processed_data_path = f'../data/training_data_with_faults/tar/processed_service_metric_data/'
    os.makedirs(processed_data_path, exist_ok=True)
    service_data.to_csv(processed_data_path+service+'_metrics.csv', index=False)
    
    # visualization_path = f'../result/visualization/normal/'+service+'/'
    visualization_path = f'../result/visualization/with_faults/'+service+'/'
    os.makedirs(visualization_path, exist_ok=True)
    fig4 = px.scatter(service_data, x='datetime', y='rr', title=service)
    fig4.update_traces(
        dict(marker={'color': 'rgb(255, 127, 14, 1)', 'opacity': 1, 'size': 3}, mode="markers"))
    pio.write_html(fig4, file=visualization_path+'rr.html')
    
    fig4 = px.scatter(service_data, x='datetime', y='sr', title=service)
    fig4.update_traces(
        dict(marker={'color': 'rgb(0, 204, 150, 1)', 'opacity': 1, 'size': 3}, mode="markers"))
    pio.write_html(fig4, file=visualization_path+'sr.html')
    
    fig4 = px.scatter(service_data, x='datetime', y='mrt', title=service)
    fig4.update_traces(
        dict(line={'color': 'rgb(31, 119, 180, 1)', 'width': 1}, mode="lines"))
    pio.write_html(fig4, file=visualization_path+'mrt.html')
    
    fig4 = px.scatter(service_data, x='datetime', y='count', title=service)
    fig4.update_traces(
        dict(line={'color': 'rgb(0, 0, 0, 1)', 'width': 1}, mode="lines"))
    pio.write_html(fig4, file=visualization_path+'count.html')


  0%|          | 0/11 [00:00<?, ?it/s]